# English tense learning

Assisted learning for english tenses.

In [ ]:
import os
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv(override=True)
api_key = os.getenv("OPENAI_API_KEY")
openai = OpenAI()

In [ ]:
def callGptMini(messages, model_override=None):
    # model = "gpt-4o-mini"
    model = "gpt-5-mini"

    if model_override is not None:
        model = model_override

    response = openai.chat.completions.create(model=model, messages=messages)
    return response.choices[0].message.content.strip()

## Sentence generation instructions

In [ ]:
def composeSystemSentenceGenerationInstructions(number_of_sentences=3):
    return f"""You are an English teacher who helps students improve their english.

Your task is to generate {number_of_sentences} sentence in user's language.
The sentences should be in the specified tense and follow the given theme.
The sentences should be of varying complexity, from simple to complex.
"""


def composeUserSentenceGenerationInstructions(language, tense, theme="any"):
    return f"""
  Language: {language}
  Tense: {tense}
  Theme: {theme}
"""


print(
    "Example:",
    composeUserSentenceGenerationInstructions("pl", "present perfect", "travel"),
)

Define an easy tense+theme combo

In [ ]:
# should be attained by user input
language = "pl"
tense = "past perfect"
theme = "travel"

In [ ]:
sentence_generation_messages = [
    {"role": "system", "content": composeSystemSentenceGenerationInstructions(3)},
    {
        "role": "user",
        "content": composeUserSentenceGenerationInstructions(language, tense, theme),
    },
]

Generate sentences to translate.

In [ ]:
sentences_to_translate = callGptMini(sentence_generation_messages)
print(sentences_to_translate)
# we could split the sentences into a list here if needed

Let's simulate the user and translate it with some super cheap model.

In [ ]:
def composeUserTranslationInstructions(
    sentences, source_language, target_language, tense
):
    return f"""
  Translate the following sentences from {source_language} to {target_language}.
  The translations should use {tense} tense.

  Sentences:
  {sentences}
  """


translation_messages = [
    {
        "role": "system",
        "content": "You are a helpful assistant that translates sentences accurately.",
    },
    {
        "role": "user",
        "content": composeUserTranslationInstructions(
            sentences_to_translate, language, "english", tense
        ),
    },
]

translations = callGptMini(translation_messages, model_override="gpt-5-nano")

print("Simulated translations:\n", translations)

Create the prompts to validate user's translations.

In [ ]:
def composeValidationSystemPrompt(user_language, tense, original_sentences):
    return f"""You are an English teacher who reviews students' sentences for grammar and vocabulary accuracy.
User's task was to translate sentences from {user_language} into English using {tense} tense.

Your task is to review the translations provided by the student.

## Guidelines
	- The sentences are specifically meant to be in {tense} tense.
  - For each sentence, provide feedback on grammar and vocabulary.
  - If the sentence is correct, respond with "Correct" and optionally provide some feedback.
  - If there are mistakes, provide the corrected sentence along with an explanation of the errors.
  - Keep your feedback concise and focused on the most important issues.
  - Verb tense usage is especially important; ensure the translations correctly use {tense} tense.

The original sentences were:
{original_sentences}

Now, analyze and review the user's translations.
"""


def composeValidationUserPrompt(user_translations):
    return f"""
  My translations:
  {user_translations}
  """


print(
    "Example:",
    composeValidationSystemPrompt("pl", tense, "sentence1\nsentence2\nsentence3"),
)
print(
    "Example:",
    composeValidationUserPrompt("My translation1\nMy translation2\nMy translation3"),
)

Finally, lets evaluate the user's effort.

In [ ]:
validation_messages = [
    {
        "role": "system",
        "content": composeValidationSystemPrompt(
            language, tense, sentences_to_translate
        ),
    },
    {"role": "user", "content": composeValidationUserPrompt(translations)},
]

validation_feedback = callGptMini(validation_messages, model_override="gpt-5-mini")
print("🗒️ Validation feedback:\n", validation_feedback)

## Summary

The workflow works fine. One thing to improve the project would be to enforce strict format. 

For example, the generated sentences should be numbered from 1 to X.
Each point (sentence) should start in a new line.
The response should not contain anything except those X sentences.

We could take the language that user is learning a parameter.

We should check list of languages that the mode knows and do not allow other languages.